Regressions between ERA5 (dif. variables) and Cloud Index (CI) in Corrientes Argentina

Preprocessing:
1) Generate detrended anomalies of obs data: CI
2) Generate detrended anomalies of grid data: ERA5 and ISCCP

1--------------------------
Las anomalias de CI se generaron asi:

* Calculo el CI mensual para los meses con menos del 20% de datos faltantes. Está en: "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
* Calculo la media regional
* Calculo la tendencia de theil-sen de la media regional
* Calculo las anomalias de CI (le resto a las medias mensuales la CI climatologica de cada mes calculada sobre el periodo comun con el satelite: fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15")
* La serie se guarda en csvs: "/pikachu/datos/nadia.testani/Doctorado/datos/smn/CI_detrend_anom_ref198312to201611.csv"

2----------------------------
ERA5 for Southamerica

3----------------------------
Cloud amount ISCCP for Southamerica

4----------------------------
ERA5 Southern Hemisphere

-----------------------------

5---------------------------
SST

In [1]:
import os
import pandas as pd 
import datetime 
import numpy as np

import iris
import iris.coord_categorisation

import scipy
from scipy.stats import theilslopes #theil-sen trends
from scipy.stats import linregress

import f_Iris

In [2]:
# directories
cwd = os.getcwd()
print("Current working directory: {0}".format(cwd))

#si estoy trabajando desde el servidor vegeta
ruta_a_archivos_ERA5_mensual = "/pikachu/datos/nadia.testani/Doctorado/datos/ERA5/mon/SAm" #ruta donde estan los archivos con datos de ERA5
ruta_a_archivos_ERA5_mensual_ori = "/datos/ERA5_updated/mon" #ruta donde estan los archivos con datos de ERA5 originales
ruta_a_archivos_CI = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2021/nubosidad/paper"
ruta_a_archivos_smn = "/pikachu/datos/nadia.testani/Doctorado/datos/smn"
ruta_a_archivos_ISCCP = "/pikachu/datos2/isccp-h/HGM"
ruta_a_resultados = "/pikachu/datos/nadia.testani/Doctorado/resultados/resultados2024" #ruta donde se guardan los resultados del analisis de nubosidad
ruta_a_archivos_SST = "/datos/OBS/SST/ERSSTV5" #en vegeta

Current working directory: /home/nadia.testani/DoctoradoCodigos/codigos2024


In [3]:
#functions

#filter theil-sen trend
def calculate_theilsen_slope(y):
    slope, intercept, _, _ = theilslopes(y, range(len(y)), alpha=0.95)
    return slope, intercept

#estimate anomalies
def f_serie_anomalias_mensuales(serie, serie_climatologia):
    serie_anomalia = serie.copy()
    for fecha in serie_anomalia.index:
        for mes in serie_anomalia.index.month.unique():
            if fecha.month == mes:
                serie_anomalia.loc[fecha] = serie.loc[fecha]-serie_climatologia.loc[mes]
    return serie_anomalia

In [4]:
#clim dates
fecha_inicio_climatologia_str = "1983-12-15"
fecha_final_climatologia_str = "2016-11-15"

In [5]:
# 1.1) Cloud Index
CI = pd.read_csv(f"{ruta_a_archivos_CI}/TCC_monthly_CI_SMN.csv") #from 1961-01-01 until 2021-11-01

#set dates as index
CI = CI.set_index(CI["Unnamed: 0"]) 
CI = CI.drop(["Unnamed: 0"], axis = 1)
CI.index = pd.DatetimeIndex(CI.index)

#estimate regional mean
CI_regional_mean = CI.mean(axis=1)

#detrend
CI_detrend = CI_regional_mean.copy()
trend = calculate_theilsen_slope(CI_detrend.values)[0] * np.arange(len(CI_detrend)) + calculate_theilsen_slope(CI_detrend.values)[1]
CI_detrend = CI_detrend - trend

#estimate climatology
CI_detrend_fechas_climatologia = CI_detrend.loc[fecha_inicio_climatologia_str:fecha_final_climatologia_str]
CI_detrend_climatologia = CI_detrend_fechas_climatologia.groupby(CI_detrend_fechas_climatologia.index.month).mean() 

#estimate anomalies
CI_detrend_anom = f_serie_anomalias_mensuales(CI_detrend, CI_detrend_climatologia)

#save
CI_detrend_anom.to_csv(f"{ruta_a_archivos_smn}/CI_detrend_anom_ref198312to201611.csv")

#CI dates
date_i_CI = CI_detrend_anom.index[0]
date_f_CI = CI_detrend_anom.index[-1]

In [9]:
# 2.1) ERA5 2-meters temperature 
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "new_data.nc"
variable_name = 't2m'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 
cube = cube[:,0,:,:] #select expver=1

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [10]:
# 2.2) ERA5 total precipitation
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "new_data.nc"
variable_name = 'tp'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 
cube = cube[:,0,:,:] #select expver=1

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [11]:
# 2.3) ERA5 mean top net long radiation flux (W*m-2)
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "new_data.nc"
variable_name = 'mtnlwrf'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 
cube = cube[:,0,:,:] #select expver=1

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [12]:
# 2.4) ERA5 vertically integrated moisture divergence (kg m**-2)
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "new_data.nc"
variable_name = 'mvimd'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 
cube = cube[:,0,:,:] #select expver=1

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [13]:
# 2.5) ERA5 omega 
file_dir = ruta_a_archivos_ERA5_mensual_ori
file_name = "ERA5_w700.nc"
variable_name = 'w'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(
            longitude=(-87, -33),
            latitude=(-60, 15),
            ignore_bounds=True)

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [14]:
# 2.6) ERA5 q850
file_dir = ruta_a_archivos_ERA5_mensual_ori
file_name = "ERA5_q850.nc"
variable_name = 'q'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(
            longitude=(-87, -33),
            latitude=(-60, 15),
            ignore_bounds=True)

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [6]:
# 2.6.1) ERA5 q850*u 
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "ERA5_q_u_850.nc"
variable_name = "q_u"
cube = iris.load_cube(f"{file_dir}/{file_name}") 

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
# 2.6.2) ERA5 q850*v 
file_dir = ruta_a_archivos_ERA5_mensual
file_name = "ERA5_q_v_850.nc"
variable_name = "q_v"
cube = iris.load_cube(f"{file_dir}/{file_name}") 

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [7]:
# 2.7) ISCCP-H monthly cloud amount --> this is done with the climatology period 

file_dir = ruta_a_archivos_ISCCP
file_name = "media_mensual_cldamt_cldamttypes.nc"
variable_name = "cldamt"
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(
            longitude=(-87, -33),
            latitude=(-60, 15),
            ignore_bounds=True)

#select dates: climatology dates
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_Climdates_detrend = f_Iris.detrend_theil_sen_cube(cube_Climdates, dimension='time', method='linear')
cube_Climdates_detrend.data[:,4,5]
iris.save(cube_Climdates_detrend, f"{ruta_a_archivos_ISCCP}/{variable_name}_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_Climdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_Climdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_Climdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_Climdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ISCCP}/{variable_name}_detrend_anoms.nc")

/home/nadia.testani/miniforge3/envs/base_nadia2/lib/python3.9/site-packages/iris/fileformats/_pyke_rules/compiled_krb/fc_rules_cf_fc.py:2190: UserWarning: Ignoring netCDF variable 'lev_3' invalid units 'level'
  warnings.warn(msg)
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
# 3.1) ERA5 geopot. height Southern Hemisphere 200 hPa
file_dir = ruta_a_archivos_ERA5_mensual_ori
file_name = "ERA5_zg.nc"
variable_name = 'z'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(longitude = (-180,180),
            latitude=(-90, 15),
            ignore_bounds=True)

#200 
cube = cube[:,0,:,:] #200 hpa

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_200hPa_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_200hPa_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [19]:
# 3.2) ERA5 geopot. height Southern Hemisphere 500 hPa
file_dir = ruta_a_archivos_ERA5_mensual_ori
file_name = "ERA5_zg.nc"
variable_name = 'z'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(longitude = (-180,180),
            latitude=(-90, 15),
            ignore_bounds=True)

#500 
cube = cube[:,1,:,:] #500 hpa

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_500hPa_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_500hPa_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


RuntimeError: NetCDF: Not a valid ID

In [18]:
# 3.3) ERA5 geopot. height Southern Hemisphere 850 hPa
file_dir = ruta_a_archivos_ERA5_mensual_ori
file_name = "ERA5_zg.nc"
variable_name = 'z'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(longitude = (-180,180),
            latitude=(-90, 15),
            ignore_bounds=True)

#850 
cube = cube[:,2,:,:] #850 hpa

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_850hpa_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_850hPa_detrend_anoms.nc")


/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [16]:
# 3.4) SST NOAA Extended Reconstructed SST V5 NOAA ERSSTv5 (in situ only)
file_dir = ruta_a_archivos_SST
file_name = "sst.mnmean.nc"
variable_name = 'sst'
cube = iris.load_cube(f"{file_dir}/{file_name}", variable_name) 

#select region
cube = cube.intersection(longitude = (-180,180),
            latitude=(-90, 15),
            ignore_bounds=True)

#select dates: common period with CI
cube_CIdates = f_Iris.clip_timerange(cube, f"{str(date_i_CI)[0:10]}/{str(date_f_CI)[0:10]}")
cube_Climdates = f_Iris.clip_timerange(cube, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

#detrend
cube_CIdates_detrend = f_Iris.detrend_theil_sen_cube(cube_CIdates, dimension='time', method='linear')
cube_CIdates_detrend.data[:,4,5]
iris.save(cube_CIdates_detrend, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_850hpa_detrend.nc")

#anomalies 
cube_Climdates_detrend = f_Iris.clip_timerange(cube_CIdates_detrend, f"{str(fecha_inicio_climatologia_str)[0:10]}/{str(fecha_final_climatologia_str)[0:10]}")

iris.coord_categorisation.add_month(cube_CIdates_detrend, 'time', name='month')
iris.coord_categorisation.add_month(cube_Climdates_detrend, 'time', name='month')

monthly_means = cube_Climdates_detrend.aggregated_by('month', iris.analysis.MEAN)

reps = (cube_CIdates_detrend.shape[0]+1)/12  #sum 1 because the last year is not complete
clim_array = np.repeat(monthly_means.data, reps, axis=0)
clim_array = clim_array[0:cube_CIdates_detrend.shape[0],:,:]
cube_detrend_anoms = cube_CIdates_detrend - clim_array
iris.save(cube_detrend_anoms, f"{ruta_a_archivos_ERA5_mensual}/{variable_name}_detrend_anoms.nc")


/home/nadia.testani/miniforge3/envs/base_nadia2/lib/python3.9/site-packages/iris/fileformats/_pyke_rules/compiled_krb/fc_rules_cf_fc.py:1940: UserWarning: Skipping global attribute 'climatology': 'climatology' is not a permitted attribute
  warnings.warn(msg.format(attr_name, str(e)))
/home/nadia.testani/miniforge3/envs/base_nadia2/lib/python3.9/site-packages/iris/fileformats/_pyke_rules/compiled_krb/fc_rules_cf_fc.py:1940: UserWarning: Skipping global attribute 'climatology': 'climatology' is not a permitted attribute
  warnings.warn(msg.format(attr_name, str(e)))
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/nadia.testani/.local/lib/python3.9/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
